In [11]:
import torch
from PIL import Image
import os

In [22]:
NO_PATH = "./cloudstreet/no/"
YES_PATH = "./cloudstreet/yes/"
TRAIN_NO_PATH = "./dataset/train/0/"
TRAIN_YES_PATH = "./dataset/train/1/"
TEST_NO_PATH = "./dataset/test/0/"
TEST_YES_PATH = "./dataset/test/1/"
train_no_num = 200
train_yes_num = 400
test_no_num = 100
test_yes_num = 200

In [23]:
lst_no = os.listdir(NO_PATH)
lst_yes = os.listdir(YES_PATH)

In [24]:
import random
datalst_no = random.sample(lst_no, k=(train_no_num+test_no_num))
datalst_yes = random.sample(lst_yes, k=(train_yes_num+test_yes_num))

In [25]:
for i in os.listdir(TRAIN_NO_PATH):
    try:
        os.remove(TRAIN_NO_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TRAIN_NO_PATH+i, e.strerror))
        
for i in os.listdir(TRAIN_YES_PATH):
    try:
        os.remove(TRAIN_YES_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TRAIN_YES_PATH+i, e.strerror))
        
for i in os.listdir(TEST_NO_PATH):
    try:
        os.remove(TEST_NO_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TEST_NO_PATH+i, e.strerror))
        
for i in os.listdir(TEST_YES_PATH):
    try:
        os.remove(TEST_YES_PATH+i)
    except OSError as e:
        print("Error: %s : %s" % (TEST_YES_PATH+i, e.strerror))


In [26]:
for i, n in enumerate(datalst_no):
    img = Image.open(NO_PATH+n).resize((800,800))
    if i < train_no_num:
        img.save(TRAIN_NO_PATH+n)
    else:
        img.save(TEST_NO_PATH+n)
    
for i, n in enumerate(datalst_yes):
    img = Image.open(YES_PATH+n).resize((800,800))
    if i < train_yes_num:
        img.save(TRAIN_YES_PATH+n)
    else:
        img.save(TEST_YES_PATH+n)

In [27]:
print(len(os.listdir(TEST_NO_PATH)), len(os.listdir(TEST_YES_PATH)), len(os.listdir(TRAIN_NO_PATH)), len(os.listdir(TRAIN_YES_PATH)))

100 200 200 400


In [32]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import csv
from model.twolayer import ConvNet

In [29]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
num_classes = 2
batch_size = 64
learning_rate = 0.001

# MNIST dataset

train_dataset = torchvision.datasets.ImageFolder(root='./dataset/train/',
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.ImageFolder(root='./dataset/test/',
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [30]:
len(train_loader)

10

In [15]:
# Convolutional neural network (two convolutional layers)
# class ConvNet(nn.Module):
#     def __init__(self, num_classes=10):
#         super(ConvNet, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=6, stride=2, padding=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=4),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.layer3 = nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
#         self.fc1 = nn.Linear(51*51*64, 64)
#         self.fc2 = nn.Linear(64, 2)
#         self.fc3 = nn.Sigmoid()
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         #print(out.size())
#         out = out.reshape(out.size(0), -1)
#         out = self.relu(self.fc1(out))
#         out = self.dropout(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         return out



In [14]:

if 'loss.csv' in os.listdir('./'):
    os.remove('loss.csv')

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        with open(r'loss.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([loss.item()])
    torch.save(model.state_dict(), 'model.ckpt')

# Test the model

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

torch.Size([64, 2])
Epoch [1/40], Step [1/10], Loss: 0.8840
torch.Size([64, 2])
Epoch [1/40], Step [2/10], Loss: 387.5888
torch.Size([64, 2])
Epoch [1/40], Step [3/10], Loss: 261.3987
torch.Size([64, 2])
Epoch [1/40], Step [4/10], Loss: 103.2427
torch.Size([64, 2])
Epoch [1/40], Step [5/10], Loss: 376.4947
torch.Size([64, 2])
Epoch [1/40], Step [6/10], Loss: 192.4856
torch.Size([64, 2])
Epoch [1/40], Step [7/10], Loss: 61.6175
torch.Size([64, 2])
Epoch [1/40], Step [8/10], Loss: 198.2867
torch.Size([64, 2])
Epoch [1/40], Step [9/10], Loss: 195.4205
torch.Size([6, 2])
Epoch [1/40], Step [10/10], Loss: 105.0928
torch.Size([64, 2])
Epoch [2/40], Step [1/10], Loss: 168.2166
torch.Size([64, 2])
Epoch [2/40], Step [2/10], Loss: 124.2459
torch.Size([64, 2])
Epoch [2/40], Step [3/10], Loss: 77.1104
torch.Size([64, 2])
Epoch [2/40], Step [4/10], Loss: 65.8472
torch.Size([64, 2])
Epoch [2/40], Step [5/10], Loss: 90.8445
torch.Size([64, 2])
Epoch [2/40], Step [6/10], Loss: 119.0965
torch.Size([64

In [33]:
model = ConvNet(num_classes)
model.load_state_dict(torch.load('model.ckpt'))
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        print("=============")
        print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(test_no_num+test_yes_num,100 * correct / total))

torch.Size([64, 1280000])
tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([64, 1280000])
tensor([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([64, 1280000])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1